In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
from collections import defaultdict

import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [2]:
input_text = au.read_txt_file_lines('input.txt')
n_rows = len(input_text)
n_cols = len(input_text[0])
# for ii in range(1, n_rows):
#     assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

input has 1000 rows and 15 cols


In [3]:
points = np.array([np.array([int(x) for x in k.split(',')]) for k in input_text])
n_points = len(points)

def dist_3d(x, y):
    '''For this exercise sqrt isn't needed'''
    return np.sum(np.power(x - y, 2))
    # return np.sqrt(np.sum(np.power(x - y, 2)))

dist_mat = np.array([[dist_3d(x, y) for x in points] for y in points])

In [4]:
n_connections = 1000 
dist_mat[np.arange(n_points), np.arange(n_points)] = np.max(dist_mat)
graph = np.zeros_like(dist_mat)
graph[dist_mat <= np.sort(np.ravel(dist_mat))[2 * n_connections - 1]] = 1
assert np.sum(graph) == 2 * n_connections

In [5]:
clusters = []
from collections import deque 
points_unassigned = set(range(n_points))

while len(points_unassigned) > 0:
    p = points_unassigned.pop()
    new_cluster = set()
    queue = deque([p])
    
    while len(queue) > 0:
        q = queue.popleft()
        if q in new_cluster:
            continue 
        new_cluster.add(int(q))
        if q in points_unassigned:
            points_unassigned.remove(q)
        for u in np.where(graph[q])[0]:
            queue.append(u)
        
    clusters.append(new_cluster)

size_clusters = sorted([len(c) for c in clusters])[::-1]
print(size_clusters[0] * size_clusters[1] * size_clusters[2])

127551


## part 2

In [6]:
dist_mat = np.array([[dist_3d(x, y) for x in points] for y in points])

## Kind of by accident I ended up doing this 2 ways:

1. Slow (10 sec): full graph check
2. Pretty fast (0.3 check): merge cluster sets

In [ ]:
points_sorted = []
for i_r, r in enumerate(dist_mat):
    for i_c, c in enumerate(r[:i_r + 1]):
        points_sorted.append((int(c), i_r, i_c))

points_sorted = sorted(points_sorted)
graph = np.zeros((n_points, n_points), dtype=int)
graph[np.arange(n_points), np.arange(n_points)] = 1

ii = 0
max_iter = 10000
best_est = 0

connections = {k: set([k]) for k in range(n_points)}
points_visited = set() 
keep_going = True 
while keep_going:
    _, i_r, i_c = points_sorted[ii]
    graph[i_r, i_c] = 1
    graph[i_c, i_r] = 1
    
    connections[i_c].add(i_r)
    connections[i_r].add(i_c)

    if i_c in points_visited and i_r in points_visited:
        pass 
    else:
        ## Go through full graph and check how many points were visited by the end
        points_visited = set() 
        queue = deque([i_c, i_r])

        while len(queue) > 0:
            p = queue.popleft()
            points_visited.add(p)
            for q in connections[p]:
                if q not in points_visited:
                    queue.append(q) 

        if len(points_visited) == n_points:
            print('got it')
            keep_going = False
            break
        elif len(points_visited) > best_est:
                # print(ii, best_est)
                best_est = len(points_visited)

    ii += 1
    if ii > max_iter:
        print('max iter')
        break 

print(points[i_r, 0] * points[i_c, 0])




got it
2347225200


In [ ]:
points_sorted = []
for i_r, r in enumerate(dist_mat):
    for i_c, c in enumerate(r[:i_r + 1]):
        points_sorted.append((int(c), i_r, i_c))

points_sorted = sorted(points_sorted)
graph = np.zeros((n_points, n_points), dtype=int)
graph[np.arange(n_points), np.arange(n_points)] = 1

ii = 0
max_iter = 10000
best_est = 0

clusters = [] 
keep_going = True 

while keep_going:
    _, i_r, i_c = points_sorted[ii]
    graph[i_r, i_c] = 1
    graph[i_c, i_r] = 1
    
    ## Merge clusters (if needed) of the two points
    ind_clusters_merge = []
    for ind_c, c in enumerate(clusters):
        if i_r in c or i_c in c:
            ind_clusters_merge.append(ind_c)
    
    if len(ind_clusters_merge) == 0:
        clusters.append(set([i_r, i_c]))
    elif len(ind_clusters_merge) == 1:
        clusters[ind_c].add(i_r)
        clusters[ind_c].add(i_c)
    elif len(ind_clusters_merge) == 2:
        new_c = clusters[ind_clusters_merge[0]].union(clusters[ind_clusters_merge[1]])
        clusters = [c for ind_c, c in enumerate(clusters) if ind_c not in ind_clusters_merge]
        clusters.append(new_c)
        if len(new_c) == n_points:
            keep_going = False
    else:
        ValueError

    ii += 1


print(points[i_r, 0] * points[i_c, 0])




2347225200
